# Decorators
This tutorial introduces decorators. A decorator _wraps_ a given function or class to modify its behavior. The vast majority of use cases are to modify functions, rather than classes, so we will concentrate on that use case in this tutorial. Until we explicitly say otherwise, we assume in the following discussion that our decorators are wrapping functions, not classes.

When a decorator wraps a function, it creates a new function that executes some code (the prologue), then calls the original function, then executes some additional code (the epilogue). Since the call to the original function is sandwiched between the prologue code and the epilogue code, we say that original function has been _wrapped_.

# A decorator example--debugging instrumentation

The following example will help illustrate the kinds of situations in which we desire something like a decorator. Suppose we want to instrument some of the functions in a particular module by adding debugging logic that tells us when such a function has been entered and tells us when it has been exited. To keep the example simple, suppose our module consists of just these two functions:

In [1]:
# Functions that construct greeting and parting sentences.

# Construct a greeting sentence
def make_greeting(name):
    return "Hello, " + name
    
# Construct a parting sentence.
def make_parting(name):
    return "Goodbye, " + name
    
# Run example
print(make_greeting('Josh'))
print(make_parting('Josh'))

Hello, Josh
Goodbye, Josh


## A solution with problems--put the debugging logic in-line
If we want to add debugging logic to these functions, we can change them to something like this.

In [2]:
# Functions that construct greeting and parting sentences.

# Construct a greeting sentence
def make_greeting(name):
    print("Enter make_greeting, arg =", name)
    result = "Hello, " + name
    print(result, "<-- Exit make_greeting")
    
    return result
    
# Construct a parting sentence.
def make_parting(name):
    print("Enter make_greeting, arg =", name)
    result = "Goodbye, " + name
    print(result, "<-- Exit make_parting")
    
    return result
    
# Run example
print(make_greeting('Josh'))
print()
print(make_parting('Josh'))

Enter make_greeting, arg = Josh
Hello, Josh <-- Exit make_greeting
Hello, Josh

Enter make_greeting, arg = Josh
Goodbye, Josh <-- Exit make_parting
Goodbye, Josh


The above solution is sub-optimal for several reasons:

  - The code necessary to instrument the functions is the same in all cases. It is "boiler-plate" code, or template-code. To implement it for a new function, we have to remember the exact template, or we risk implementing it incorrectly.
  - If we decide in the future to change how the debugging works, we would have to make changes all over the code base.
  - The debugging code is orthogonal to the central purposes of the functions it modifies, obscuring these purposes and making the code harder to understand.
  
Ideally, we would like to isolate the code that handles the debugging logic in a single place and have it not be visible in the functions that use it. If we did so, this would immediately remove all three problems above.

This is the reason we desire decorators. A decorator is usually created because it represents a general-purpose modification that could be used on many different functions, irrespective of the central purpose of such a function. Applying the decorator to the function endows the function with the enhancement provided by the decorator, and it does so without cluttering the code of the function or requiring the coder to remember the details required to implement a specific code template.

## An "old-school" solution--no decorators

As it turns out, we don't really need decorators to achieve all of the above in python. Below we illustrate how to wrap a function "old school", without the use of a decorators.

In [3]:
# Define a function that implements the debugging logic. It will "wrap" the
# desired function.

def debug(f):
    """
    Return a wrapped function that implements debugging
    logic for the function f.
    """
    def wrapper(*args, **kwds):
        # prologue
        print("Enter", f.__name__, end = '')
        print(", args = ", args, sep = '', end = '')
        print(", kwds =", kwds)
        
        result = f(*args, **kwds) # (3) Call the original function.
        
        # epilogue
        print(result, "<-- Exit", f.__name__)

        return result
    
    return wrapper


# Construct a greeting sentence
def make_greeting(name):
    return "Hello, " + name
make_greeting = debug(make_greeting) # (4) We wrap make_greeting() with debugging logic

# Construct a parting sentence
def make_parting(name):
    return "Goodbye, " + name
make_parting  = debug(make_parting)  # (5) We wrap make_parting() with debugging logic
 

# Run example
print(make_greeting('Josh'))
print()
print(make_parting('Josh'))

Enter make_greeting, args = ('Josh',), kwds = {}
Hello, Josh <-- Exit make_greeting
Hello, Josh

Enter make_parting, args = ('Josh',), kwds = {}
Goodbye, Josh <-- Exit make_parting
Goodbye, Josh


## An elegant solution--with decorators
The code above achieves everything we would like. The debugging logic is safely put in a single place, and it no longer clutters the definitions of `make_greeting()` or `make_parting()`. The only slight inelegance is that we have to bash the definitions of `make_greeting()` and `make_parting()` in (4) and (5) above. Python introduced decorators in order to make these lines unnecessary. A _decorator_ therefore is really just _syntactic sugar_ that makes it easier and more elegant to wrap a function. Here is the same code again, using decorators.

In [4]:
# Apply debug as a decorator to add debugging logic.

# We omit the definition of debug(). 
# We don't need to alter a single line of its code, 
# as defined above in [3].

# Construct a greeting sentence
@debug                    # (6) Here we employ debug() as a decorator.
def make_greeting(name):  # (7) This is the function definition that is decorated.
    return "Hello, " + name

# Construct a parting sentence
@debug                    
def make_parting(name):
    return "Goodbye, " + name
 

# Run example
print(make_greeting('Josh'))
print()
print(make_parting('Josh'))    

Enter make_greeting, args = ('Josh',), kwds = {}
Hello, Josh <-- Exit make_greeting
Hello, Josh

Enter make_parting, args = ('Josh',), kwds = {}
Goodbye, Josh <-- Exit make_parting
Goodbye, Josh


# Decorators Defined
For the moment, we will define a decorator as a function that takes a single argument, which must also be a function. When a decorator is called, it returns as its value yet another function that is a wrapped version of the input argument. (The true definition of a decorator in python is slightly more general. We'll discuss that later.)

In the above code, on the line marked (6), the presence of the `@` sign before `debug` tells python to treat `debug` as a decorator. As such, it expects `debug` to be a function that takes a single argument that is also a function. It also expects there to be a function definition directly beneath the decorator. We say in this case that `debug` _decorates_ `make_greeting`. The python interpeter evaluates the function definition on (7), resulting in a function object, and then calls `debug` on this function object, assigning the return value of `debug` to the global `make_greeting`. In order words, `debug` gets fed in the original definition of `make_greeting` and returns a wrapped version of it, which wrapped version is the one actually assigned to the global `make_greeting`.

# Decorators are quite useful
It may seem as though it isn't worth learning about something as fancy as decorators in order to implement something as simple as the above debugging logic; however, keep in mind that we intentionally keep examples simple here so that they will be more understandable.

In reality, a decorator for debugging would have to handle exceptions and would also have to perform some kind of indenting to keep track of call levels. Below is an updated decorator that does these things. As one can see, it is well worth isolating all the debugging logic within a single function. The single line marked (8) below is wrapped by a significant amount of prologue and epilogue code.

# A more realistic version of the debugging decorator

In [5]:
# Define a decorator that adds debugging logic to any function.

DebugCallLevel = -1 # Tracks how many nested calls have been made.
DEBUG_INDENT   = 4  # How many spaces to lead with for each call-level.


def debug(f):
    """
    This is a decorator. Enhance the function f
    so that it prints out debugging information
    about when it is called and what results it returns.
    """
    def wrapper(*args, **kwds):
        global DebugCallLevel
        
        # We need a try/except block to properly handle errors.
        try:
            # ========== Prologue
            
            DebugCallLevel += 1
            
            # Provide information about entering the function.
            print(' '*4*DebugCallLevel, "Enter", f.__name__, end = '')
            print(", args = ", args, sep = '', end = '')
            print(", kwds =", kwds)
            
            # ==========
            
            result = f(*args, **kwds) # (8) Here we call the main function.
            
        # ========== Epilogue
        
        # We got an error-- provide information about it and re-raise it.
        except Exception as e:
            print(' '*DEBUG_INDENT*DebugCallLevel, "Debug: Encountered an error while evaluating", f.__name__)
            print(' '*DEBUG_INDENT*DebugCallLevel, "  Error:", e)
            if DebugCallLevel > 0:
                raise e
                
        # No error occurred--provide info about exiting the function.
        else:
            print(' '*DEBUG_INDENT*DebugCallLevel, result, "<-- exit", f.__name__)

            return result
        
        finally:
            DebugCallLevel -= 1
    
    return wrapper


# Calculate the factorial function recursively.
# Used as a test case for our debug decorator.
@debug
def factorial(n):
    if n <= 0:
        return 1.0
    else:
        return n * factorial(n - 1)


# Run example
print(factorial(3))
print()
factorial('hello')
    

 Enter factorial, args = (3,), kwds = {}
     Enter factorial, args = (2,), kwds = {}
         Enter factorial, args = (1,), kwds = {}
             Enter factorial, args = (0,), kwds = {}
             1.0 <-- exit factorial
         1.0 <-- exit factorial
     2.0 <-- exit factorial
 6.0 <-- exit factorial
6.0

 Enter factorial, args = ('hello',), kwds = {}
 Debug: Encountered an error while evaluating factorial
   Error: '<=' not supported between instances of 'str' and 'int'


# Decorator Generators
The expression that follows the `@` sign doesn't have to be a function name. It can also be a function call, so long as the object returned by the function call is a decorator. This allows us to define functions that are not decorators in themselves, but which generate decorators. Read the example below, which alters the `debug` function defined above so that it is no longer a decorator but becomes a decorator generator.

In [6]:
# Define a decorator generator that adds debugging logic to any function.

# Only print out debugging information beneath
# a certain call-level. Once the call-level gets too deep,
# debugging information is no longer printed out.

DebugCallLevel = -1 # Tracks how many nested calls have been made.
DEBUG_INDENT   = 4  # How many spaces to lead with for each call-level.

def debug(max_call_levels):
    """
    This is a decorator generator.
    
    Return a decorator appropriate for wrapping functions with debug information.
    
    :param max_call_levels: an integer >= 1. The number of call levels to show
                            debug info for.
    """
    def decorator(f):
        """This is a dynamically generated decorator.
    
           Enhance the function f so that it prints out debugging information
           about when it is called and what results it returns, so long as the
           call-level is less than max_call_levels.
        """
        def wrapper(*args, **kwds):
            global DebugCallLevel

            # Make sure our debug level is properly reset when any errors are encountered.
            try:
                # Prologue
                DebugCallLevel += 1

                if DebugCallLevel < max_call_levels: 
                    print(' '*4*DebugCallLevel, "Enter", f.__name__, end = '')
                    print(", args = ", args, sep = '', end = '')
                    print(", kwds =", kwds)
                    
                    
                result = f(*args, **kwds) # (8) Here we call the main function.

            # Epilogue
            except Exception as e:
                if DebugCallLevel < max_call_levels:
                    print(' '*DEBUG_INDENT*DebugCallLevel, "Debug: Encountered an error while evaluating", f.__name__)
                    print(' '*DEBUG_INDENT*DebugCallLevel, "  Error:", e)
                if DebugCallLevel > 0:
                    raise e
            else:
                if DebugCallLevel < max_call_levels:
                    print(' '*DEBUG_INDENT*DebugCallLevel, result, "<-- exit", f.__name__)

                return result
            finally:
                DebugCallLevel -= 1
        
        return wrapper
    return decorator


# Calculate the factorial function recursively.
# Used as a test case for our debug decorator.
@debug(4)
def factorial(n):
    if n <= 0:
        return 1.0
    else:
        return n * factorial(n - 1)


# Run example
print(factorial(8))
print()
factorial('hello')
    

 Enter factorial, args = (8,), kwds = {}
     Enter factorial, args = (7,), kwds = {}
         Enter factorial, args = (6,), kwds = {}
             Enter factorial, args = (5,), kwds = {}
             120.0 <-- exit factorial
         720.0 <-- exit factorial
     5040.0 <-- exit factorial
 40320.0 <-- exit factorial
40320.0

 Enter factorial, args = ('hello',), kwds = {}
 Debug: Encountered an error while evaluating factorial
   Error: '<=' not supported between instances of 'str' and 'int'


In the example above, one can see that there are now three nested levels of function definition instead of just two, as we had before. This can seem a little bit mind-boggling, and it's easy to get lost among the nested levels. We'll take them one at a time.

The outermost function, `debug`, is no longer a decorator. Instead it is a decorator _generator_, which means it must return a decorator when called. It does that, by returning the function named `decorator`. This function is a decorator, because it takes a function `f` as an argument and returns the function `wrapper` as its return value. The function `decorator` is really just the same as the function we named `debug` in the previous example. But, by adding one more level of indirection, we can now pass in the `max_call_levels` variable to `debug` and have that imbedded in the closure that results when we return `wrapper`.

# Odds And Ends
We have presented examples above that illustrate the main use case for decorators, which includes decorator generators. As mentioned in the first paragraph of this tutorial, decorators are actually a little bit more general than we have described, although the extra generality does not get used that much in practice. Here we describe the additional contexts in which decorators may be used.

## Decorators can return any object at all
To begin with, a decorator need not return a function. It can return any python object at all. In particular, it can return a class object, which is what it should do when it is decorating a class instead of a function (this case is discussed in the next section). The next two examples are not particularly motivated or useful. They merely illustrate the freedom possible with decorators.


In [7]:
def foo(f):
    """
    This decorator ignores its input
    and returns the list [1,2,3]
    """
    return [1,2,3]

@foo
def bar (x):
    return x + 1

# Example 
# The global bar now evaluates to the list [1,2,3]
# It is not a function
print(bar)

[1, 2, 3]


In the code above, the global name `bar` now just points to the list `[1,2,3]`, rather than to a function. The original function was entirely ignored by the decorator. That is allowed, although not very useful.

Below, instead of ignoring the function, we create a list that contains it as a single element.

In [8]:
def listify(f):
    """
    This decorator puts f inside a list.
    """
    return [f]

@listify
def bar(x):
    return x + 1

# Example
# The global bar now evaluates to a list containing
# the original function bar.
print(bar)

[<function bar at 0x109964a60>]


Finally, below is a more motivated example. This decorator returns a callable object. It is not a function object, but it can be used like a function. Note that the example below is a general-purpose alternative to defining a wrapper function. The Counter instance behaves like a wrapper function.

In [9]:
def counter(f):
    """
    This is a decorator.
    It returns a proxy for f that tracks how many times f is called.
    """
    return Counter(f)

class Counter(object):
    """
    Represents a proxy for a function f.
    Enhances f by tracking how many times f is called.
    """
    def __init__(self, f):
        self.f = f       # The function we are a proxy for.
        self.counter = 0 # Keep track of how many times f is called.
        
    # Defining this method makes Counter instances act like functions.
    def __call__(self, *args, **kwds):
        self.counter += 1
        return self.f(*args, **kwds)
    
    def report(self):
        """
        Report how many times f has been called.
        """
        print("{name} has been called {n} times".format(name = self.f.__name__,
                                                        n = self.counter))
    
@counter
def foo(x):
    return x + 1

# Example
print(foo(11))
print(foo(20))
print(foo(50))
foo.report()
print("foo is a Counter object:", foo)
        

12
21
51
foo has been called 3 times
foo is a Counter object: <__main__.Counter object at 0x109952d30>


## Decorators can decorate class definitions
As mentioned in the first paragraph, a decorator may decorate a class definition instead of a function. Below is an example. Note that the decorator is now returning a class object instead of a function. (Note also that python has more powerful facilities for creating specialized classes, called _metaclasses_, but those will not be covered in this course.)

In [10]:
def foo(cls):
    """
    This is a class decorator.
    It gives cls a class variable called COLOR, set to 'GREEN',
    and a new method called area()
    """
    cls.COLOR = 'GREEN'  # Create a class variale

    # Add an area() method. 
    # Note: this assumes cls defines
    # width and height member variables.
    def area(self):
        return self.width * self.height
    
    cls.area = area
    
    return cls

@foo
class Rectangle(object):
    def __init__(self, width, height):
        self.width = width
        self.height = height
        
# Example
r = Rectangle(2, 3)
print("The color of r is", r.COLOR)
print("The area of r is", r.area())

The color of r is GREEN
The area of r is 6


## Decorators don't need to be functions.
Any callable object can be a decorator. Functions are simply the most common type of callable object. Below, we make instances of class `CallCounter` callable by giving them a `__call__()` method. This allows them to be decorators.

In [11]:
class CallCounter(object):
    """
    A decorator that counts how many times each function it decorates has been called.
    """
    def __init__(self):
        self.counts_dict = {} # A dictionary of function/count pairs
            
    def report(self):
        for f, count in self.counts_dict.items():
                print("{name} has been called {n} times".format(name = f.__name__,
                                                                n = count))
        
    def __call__(self, f):
        """
        Wrap the function f so that it prints out how many times it has been called
        and throws an error if it exceeds self.max_calls.
        """
        self.counts_dict[f] = 0 # Initialize counter for f
        
        def wrapper(*args, **kwds):
            self.counts_dict[f] += 1
            return f(*args, **kwds)
        
        return wrapper
            

    
count_calls = CallCounter() # Establish a call counter

@count_calls
def foo(x):
    return x + 1

@count_calls
def bar(x):
    return x + 2

print(foo(1))
print(foo(10))
print(bar(3))
print(bar(7))
print(bar(17))
count_calls.report()




2
11
5
9
19
foo has been called 2 times
bar has been called 3 times
